In [ ]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC


In [ ]:
!pip install -q kaggle


In [ ]:
from google.colab import files


In [ ]:
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhishek262","key":"f03ac5c0db1f2a506d2711e50d60e19f"}'}

In [ ]:
!mkdir ~/.kaggle


In [ ]:
! cp kaggle.json ~/.kaggle/


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
! mkdir traffic_dataset


In [ ]:
! kaggle datasets download -d silicon99/dft-accident-data


100% 112M/112M [00:04<00:00, 26.0MB/s] 



In [ ]:
! unzip dft-accident-data.zip -d traffic_dataset/


Archive:  dft-accident-data.zip
  inflating: traffic_dataset/Accidents0515.csv  
  inflating: traffic_dataset/Casualties0515.csv  
  inflating: traffic_dataset/Vehicles0515.csv  
  inflating: traffic_dataset/contextCSVs/Accident_Severity.csv  
  inflating: traffic_dataset/contextCSVs/Age_Band.csv  
  inflating: traffic_dataset/contextCSVs/Casualty_Class.csv  
  inflating: traffic_dataset/contextCSVs/Casualty_Severity.csv  
  inflating: traffic_dataset/contextCSVs/Casualty_Type.csv  
  inflating: traffic_dataset/contextCSVs/Day_of_Week.csv  
  inflating: traffic_dataset/contextCSVs/Journey_Purpose.csv  
  inflating: traffic_dataset/contextCSVs/Junction_Control.csv  
  inflating: traffic_dataset/contextCSVs/Junction_Detail.csv  
  inflating: traffic_dataset/contextCSVs/Junction_Location.csv  
  inflating: traffic_dataset/contextCSVs/Light_Conditions.csv  
  inflating: traffic_dataset/contextCSVs/Local_Authority_District.csv  
  inflating: traffic_dataset/contextCSVs/Local_Authority_Highw

In [ ]:


accidents = pd.read_csv('traffic_dataset/Accidents0515.csv',index_col='Accident_Index')
# casualties=pd.read_csv('traffic_dataset/Casualties0515.csv' , error_bad_lines=False,index_col='Accident_Index',warn_bad_lines=False)
# vehicles=pd.read_csv('traffic_dataset/Vehicles0515.csv', error_bad_lines=False,index_col='Accident_Index',warn_bad_lines=False)


In [ ]:

#dropping columns not relevant to our training set
accidents.drop(['Location_Easting_OSGR', 'Location_Northing_OSGR','LSOA_of_Accident_Location',
                'Junction_Control' ,'2nd_Road_Class'], axis=1, inplace=True)


In [ ]:
#dropping columns
accidents.drop(columns=["Police_Force","Number_of_Casualties","Date","Local_Authority_(District)","Local_Authority_(Highway)",
        '1st_Road_Number', 'Road_Type',
              'Junction_Detail', '2nd_Road_Number',
                     'Pedestrian_Crossing-Human_Control',
                            'Pedestrian_Crossing-Physical_Facilities', 
                              
                                          'Special_Conditions_at_Site', 'Carriageway_Hazards',
                                                  'Did_Police_Officer_Attend_Scene_of_Accident'
                                                         ],inplace = True)

In [ ]:
accidents.columns



Index(['Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident'],
      dtype='object')

In [ ]:
#dropping NA columns
accidents.dropna(inplace=True)
#considering only first million rows to fit KNN to memory
accidents = accidents.iloc[:1000000]
X = accidents.drop(columns=["Accident_Severity","Time"])
y = accidents["Accident_Severity"]


In [ ]:
#splitting the dataset into training, testing subsets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:

X_test.columns

Index(['Longitude', 'Latitude', 'Number_of_Vehicles', 'Day_of_Week',
       '1st_Road_Class', 'Speed_limit', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions', 'Urban_or_Rural_Area'],
      dtype='object')

In [ ]:
X_train.columns

Index(['Longitude', 'Latitude', 'Number_of_Vehicles', 'Day_of_Week',
       '1st_Road_Class', 'Speed_limit', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions', 'Urban_or_Rural_Area'],
      dtype='object')

###Building the model

In [ ]:
#Init the model using 3/5 neighbours
neigh = KNeighborsClassifier(n_neighbors=5,algorithm="ball_tree")

In [ ]:
#Train
neigh.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
print("accuracy: ",neigh.score(X_test,y_test)*100)


accuracy:  89.55


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = neigh.predict(X_test)
print(classification_report(y_true = y_test,y_pred=y_pred))

              precision    recall  f1-score   support

           2       0.12      0.01      0.02       196
           3       0.90      0.99      0.94      1804

    accuracy                           0.90      2000
   macro avg       0.51      0.50      0.48      2000
weighted avg       0.83      0.90      0.85      2000

